In [1]:
from dotenv import load_dotenv

# 토큰 정보 로드
load_dotenv()

True

In [26]:
from typing import Dict, TypedDict

from langchain_core.messages import BaseMessage
import base64


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [27]:
import json
import matplotlib.pyplot as plt
import operator
from typing import Annotated, Sequence, TypedDict

from langchain import hub
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores.chroma import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.tools.sql_database.tool import QuerySQLCheckerTool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from IPython.display import Image, display
import zlib
    
### Nodes ###
def prettier_code(state):
    """
    Just first node.
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): The current graph state
    """

    state_dict = state["keys"]
    code = state_dict["code"]
    code_type = state_dict["code_type"]
    target_code_type = state_dict["target_code_type"]

    # prompt = PromptTemplate(
    #     template="""Given user input code, your task is to reformat or "prettify" the code according to best practices for readability and maintainability. This process involves applying consistent formatting rules, such as indentation, line breaks, spacing, and organizing imports or declarations. The goal is to make the code more understandable and cleaner without altering its functionality.

    #     1. Input: Receive two pieces of information from the user - the code snippet and the programming language type.
    #     2. Identify the programming language of the input code to apply the appropriate formatting rules and conventions specific to that language.
    #     3. Analyze the code structure to identify blocks of code, such as functions, loops, conditionals, and class definitions. This will guide the application of indentation and spacing.
    #     4. Apply indentation consistently. For example, use spaces or tabs (whichever is preferred or standard for the language) to indent nested blocks of code.
    #     5. Ensure proper spacing around operators (like +, -, =, etc.), commas, and semicolons for better readability.
    #     6. Add line breaks strategically to separate logical sections of the code, such as between function definitions, or to break up long lines of code for readability.
    #     7. Organize and group related declarations and statements, such as grouping variable declarations together at the beginning of a function or sorting import statements alphabetically.
    #     8. Remove any extraneous whitespace, such as trailing spaces at the end of lines or unnecessary blank lines, while keeping logical breaks between sections.
    #     9. Optionally, add or improve comments to explain complex logic, function purposes, and any non-obvious code segments.

    #     Output: Provide the prettified version of the input code, ensuring it is formatted for optimal readability and adheres to the conventions of the programming language.

    #     Self Evaluation:
    #     - Review the prettified code to ensure that the formatting changes have not altered the code's functionality or introduced any syntax errors.
    #     - Check that the applied formatting rules are consistent throughout the code and align with best practices for the programming language.
    #     - Assess the readability of the code after prettification, considering whether the changes make it easier to understand the code's structure and logic.
    #     - Ensure that any added comments are clear, concise, and helpful in understanding the code.

    #     Re-answer based on self-evaluation results:
    #     - If inconsistencies or issues are identified during self-evaluation, adjust the formatting rules applied and correct any errors.
    #     - Refine the prettification process to better balance readability with conciseness, ensuring that the code remains maintainable and easy to navigate.
    #     - Apply lessons learned from the self-evaluation to improve the prettification technique for future use, optimizing the approach for efficiency and effectiveness in enhancing code readability.

    #     IMPORTANT: JUST PRINT OUT Code ONLY!!.

    #     Here is the code.
    #     ```
    #     {code}
    #     ```
    #     """,
    #     input_variables=["code", "code_type"],
    # )
    #     # IMPORATNT: Output only plain text. Do not output markdown.

    # # LLM
    # llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # # Chain
    # rag_chain = prompt | llm | StrOutputParser()

    # # Run
    # prettier_code = rag_chain.invoke({"code": code, "code_type": code_type})

    return {
        "keys": {"code": code, "code_type": code_type, "target_code_type": target_code_type,}
    }


def code_anlysis(state):
    """
    Make sure there is nothing wrong with the code entered by the user.

    Args:
        state (dict): The current graph state

    Returns:
        str: Descision next node to call
    """

    prompt = PromptTemplate(
        template=
        """
        "Construct a system that validates code snippets based on the specified programming language type, ensuring there are no syntax or compilation errors. This requires the system to have a comprehensive understanding of the syntax rules and error detection mechanisms specific to each programming language. The system should parse the user's code, check it against the language's syntax rules, and perform a basic static analysis to catch common errors.
        
        The programming language type is: {code_type}

        Here is the code snippet.
        ```
        {code}
        ```

        1. Input: Receive two pieces of information from the user - the code snippet and the programming language type.
        2. Based on the specified programming language, load the corresponding syntax rules and error detection criteria.
        3. Parse the code snippet to identify its structural components, such as variables, functions, loops, and conditional statements.
        4. Perform a syntax check to ensure all language constructs are correctly used according to the language's syntax rules.
        5. Conduct a static analysis to identify common errors, such as:
          - Undefined variables or functions.
          - Mismatched data types.
          - Unreachable code or infinite loops.
          - Improper use of language-specific constructs or libraries.
        6. If any errors are detected, print 'yes'
        7. If no errors are found, print 'no'.

        Self Evaluation:
        - Ensure that the system covers a comprehensive set of syntax rules and common errors for the specified programming language.
        - Check that the parsing and analysis processes are accurate and thorough, capable of catching both simple and complex errors.
        - Assess the system's flexibility and scalability, ensuring it can be easily updated or expanded to include more programming languages or additional types of error detection.


        IMPORTANT: You only print 'yes' or 'no'.

        """,
        input_variables=["code", "code_type"],
    )
  
    state_dict = state["keys"]
    code = state_dict["code"]
    code_type = state_dict["code_type"]

    # Data model
    class safe(BaseModel):
        """Check the code for grammer errors"""
        is_exist_error: str = Field(description="Supported value 'yes' or 'no'")
    
    llm = ChatOpenAI(temperature=0, model="gpt-4-0125-preview", streaming=True)

    safe_tool_oai = convert_to_openai_tool(safe)

    # LLM with tool and enforce invocation
    llm_with_tool = llm.bind(
        tools=[convert_to_openai_tool(safe_tool_oai)],
        tool_choice={"type": "function", "function": {"name": "safe"}},
    )

    # Parser
    parser_tool = PydanticToolsParser(tools=[safe])


    # Chain
    chain = prompt | llm_with_tool | parser_tool

    result = chain.invoke({"code": code, "code_type": code_type})


    safe = result[0].is_exist_error

    if safe == "no":
        print("there are no error found.")
        return "no"
    else:
        print("error existed.")
        return "no"
    
def generate_converted_code(state):
    """
    Generate a code.
    
    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates converted_code key with coverted code
    """
    state_dict = state["keys"]
    code = state_dict["code"]
    code_type = state_dict["code_type"]
    target_code_type = state_dict["target_code_type"]

    prompt = PromptTemplate(
        template="""Given the original code in a specific programming language and the target programming language, your task is to convert the code from the original language to the target language. This process requires understanding the syntax, data types, control structures, and libraries or frameworks used in both the original and target languages. 

        1. Start by analyzing the given code snippet to identify its core functionality, including variables, functions, loops, conditionals, and any library or API calls.
        2. Research the equivalent syntax, library and constructs in the target programming language. This may involve looking up documentation, tutorials, or code examples online.
        3. Begin converting the code section by section, starting with variable declarations and initialization, followed by control structures (if-else statements, loops), functions or methods, and finally, any specialized library or API usage.
        4. Pay special attention to data types and their compatibility between the two languages, as this can be a common source of bugs or errors.
        5. If there are any new variables, functions, classes, etc., use names that fit the role.
        6. After converting the code, perform a self-review to ensure the logic and functionality match the original code. Look for any syntax errors or overlooked language-specific nuances.
        7. Test the converted code with the same inputs used for the original code to verify that it produces the same outputs. Adjust the code as necessary to fix any discrepancies or errors encountered during testing.

        Self Evaluation:
        - Was the import statement of the library you used written?
        - Did the converted code compile and run without errors in the target language environment?
        - Does the converted code accurately replicate the functionality of the original code, producing the same outputs for given inputs?
        - Were there any parts of the original code that required specific consideration due to differences in the programming languages (e.g., memory management, concurrency, library support)?
        - If any bugs or errors were encountered during testing, were they addressed effectively, and what was the root cause?
        - How could the conversion process be improved for efficiency or readability of the converted code?

        Re-answer based on self-evaluation results:
        - Adjust the conversion approach if necessary, focusing on areas where the initial conversion may have missed language-specific features or introduced bugs.
        - Apply any insights gained from the self-evaluation to refine the converted code, ensuring it is as efficient, readable, and true to the original functionality as possible.
        
        
        The programming language type is: {code_type}
        The target programming language tpye is: {target_code_type}

        IMPORATNT: YOUR TURN! JUST PRINT OUT CODE.
        Here is the code snippet.
        ```
        {code}
        ```


        """,
        input_variables=["code", "code_type", "target_code_type"],
    )

    # LLM
    llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

    # Chain
    rag_chain = prompt | llm | StrOutputParser()

    # Run
    converted_code = rag_chain.invoke({"code": code, "code_type": code_type, "target_code_type": target_code_type})

    return {
        "keys": {"code": code, "code_type": code_type, "target_code_type": target_code_type, "converted_code": converted_code,}
    }

In [28]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
# workflow.add_node("prettier_code", prettier_code)  # prettier_code
workflow.add_node("generate_converted_code", generate_converted_code)  # generate_converted_code

# Build graph
# workflow.set_entry_point("prettier_code")
workflow.set_entry_point("generate_converted_code")
# workflow.add_conditional_edges(
#     "prettier_code",
#     code_anlysis,
#     {
#         "yes": END,
#         "no": "generate_converted_code",
#     },
# )
# workflow.add_edge("prettier_code", "generate_converted_code")
workflow.add_edge("generate_converted_code", END)

# Compile
app = workflow.compile()

In [ ]:
import pprint
# Run
inputs = {"keys": {
    "code_type": "python",
    "target_code_type": "typescript",
    "code": '''# Local
from langchain_community.chat_models import ChatOllama

llama2_chat = ChatOllama(model="llama2:13b-chat")
llama2_code = ChatOllama(model="codellama:7b-instruct")

# API
from langchain_community.llms import Replicate

# REPLICATE_API_TOKEN = getpass()
# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
replicate_id = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
llama2_chat_replicate = Replicate(
    model=replicate_id, input={"temperature": 0.01, "max_length": 500, "top_p": 1}
)
    '''
}}
for output in app.stream(inputs, {"recursion_limit": 5}):
    for key, value in output.items():
        # Node
        pprint.pprint(f"Node '{key}':")
        # Optional: print full state at each node
        # pprint.pprint(value["keys"], indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

pprint.pprint(value['keys']['converted_code'])

In [8]:
from langchain_community.llms.ollama import Ollama
from langchain.prompts import PromptTemplate
from langchain.chat_models.anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
  template="""Given the original code in a specific programming language and the target programming language, your task is to convert the code from the original language to the target language. This process requires understanding the syntax, data types, control structures, and libraries or frameworks used in both the original and target languages. 

  
  The programming language type is: {code_type}
  The target programming language tpye is: {target_code_type}
  Here is the code snippet.
  ```
  {code}
  ```
  1. Start by analyzing the given code snippet to identify its core functionality, including variables, functions, loops, conditionals, and any library or API calls.
  2. Research the equivalent syntax, library and constructs in the target programming language. This may involve looking up documentation, tutorials, or code examples online.
  3. Begin converting the code section by section, starting with variable declarations and initialization, followed by control structures (if-else statements, loops), functions or methods, and finally, any specialized library or API usage.
  4. Pay special attention to data types and their compatibility between the two languages, as this can be a common source of bugs or errors.
  5. If there are any new variables, functions, classes, etc., use names that fit the role.
  6. If the variable name is a reserved word in the changed language, it should be changed appropriately.
  7. After converting the code, perform a self-review to ensure the logic and functionality match the original code. Look for any syntax errors or overlooked language-specific nuances.
  8. Test the converted code with the same inputs used for the original code to verify that it produces the same outputs. Adjust the code as necessary to fix any discrepancies or errors encountered during testing.

  Self Evaluation:
  - Was the import statement of the library you used written?
  - Did the converted code compile and run without errors in the target language environment?
  - Does the converted code accurately replicate the functionality of the original code, producing the same outputs for given inputs?
  - Were there any parts of the original code that required specific consideration due to differences in the programming languages (e.g., memory management, concurrency, library support)?
  - If any bugs or errors were encountered during testing, were they addressed effectively, and what was the root cause?
  - How could the conversion process be improved for efficiency or readability of the converted code?

  Re-answer based on self-evaluation results:
  - Adjust the conversion approach if necessary, focusing on areas where the initial conversion may have missed language-specific features or introduced bugs.
  - Apply any insights gained from the self-evaluation to refine the converted code, ensuring it is as efficient, readable, and true to the original functionality as possible.
  

  IMPORATNT: YOUR TURN! JUST PRINT OUT CODE.


  """,
  input_variables=["code", "code_type", "target_code_type"],
)


code = '''main() {

  // The ?? operator returns the first expression IFF it is not null
  var monday = 'doctor';
  var tuesday;
  var next = tuesday ?? monday;
  print('next appointment: $next');

  // the ??= operator assigns a value IFF it is not null
  var wednesday;
  next ??= wednesday;
  print('next appointment: $next');

  // the ? operator calls a function IFF the object is not null
  String thursday;
  var length = thursday?.length;
  print('length: $length');

}
'''

code_type="python"
target_code_type="typescript"

# LLM
llm = ChatOpenAI(model_name="gpt-4-0125-preview", temperature=0)

# Chain
chain = prompt | llm | StrOutputParser()

# Run
result = chain.invoke({"code": code, "code_type": code_type, "target_code_type": target_code_type})

print(result)



```typescript
function main() {
  // The ?? operator returns the first expression if it is not null or undefined
  let monday: string = 'doctor';
  let tuesday: string | undefined;
  let next: string = tuesday ?? monday;
  console.log(`next appointment: ${next}`);

  // The ??= operator assigns a value if it is not null or undefined
  let wednesday: string | undefined;
  next ??= wednesday;
  console.log(`next appointment: ${next}`);

  // The ?. operator calls a method or accesses a property if the object is not null or undefined
  let thursday: string | undefined;
  let length: number | undefined = thursday?.length;
  console.log(`length: ${length}`);
}

main();
```

Explanation of the translation process:

1. **Key Functionalities and Algorithms**: The original Dart code demonstrates the use of null-aware operators (`??`, `??=`, and `?.`). These operators are used to handle null values in variables and method calls.

2. **Data Types Mapping**: Dart's `var` keyword is replaced with T